## Analizar una conversación de Whatsapp

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
df = pd.read_csv("chat_exported_english.txt", header=None, encoding='utf-8', delimiter ='qwerty')#,on_bad_lines='skip')
df.columns = ['base']


#Concatenar columnas con saltos de línea
df["log_count"] = (df["base"].str.match(r'^[0-9]')).cumsum()
print("Registros con saltos de linea: " + str(df[df["base"].str.match(r'^[0-9]')==False]["log_count"].unique()))
df = df.groupby(["log_count"], sort=False)["base"].agg(' '.join).reset_index()
print("Registros Totales: " + str(df.shape))


#Establecer el idioma en el que estaba configurado wsp
language_settings = "ingles"



Registros con saltos de linea: []
Registros Totales: (8, 2)


In [2]:
# Limpiamos aquellos registros que no tengan como remitente ninguno de los dos participantes
df["remitente"] = df["base"].str.split("-", expand=True)[1].str.split(":", expand=True)[0]
#df = df.drop(df[(df.remitente!=' Yo')&(df.remitente!=' Mi Amor')].index)


In [3]:
#Extracción de columnas base
df["fecha"] = df["base"].str.split(",", expand=True)[0]
df['fecha'] = pd.to_datetime(df['fecha'])
df["time"] = df["base"].str.split(" ", expand=True)[1]
df["mensaje"] = df["base"].str.split(":", expand=True, n=2)[2]


In [4]:
#Extracción de columnas adicionales dependiendo de la configuración
if language_settings == "español":
    df["part_day"] = df["time"].str.slice(-5)[1]
    df["minuto"] = df["time"].str.split(":", expand=True)[1].str.slice(0,2)
    df["hora"] = df["time"].str.split(":", expand=True)[0]
    #Conversión de formato de hora a 24 horas
    df.loc[(df["part_day"].str.match(r'^p')), "hora"] = df.hora.astype("int")+12
else:
    df["part_day"] = df["base"].str.split("-", expand=True)[0].str.split().str[-1]
    df["hora"] = df["time"].str.split(":", expand=True)[0]
    df["minuto"] = df["time"].str.split(":", expand=True)[1]
    #Conversión de formato de hora a 24 horas
    df.loc[(df.part_day=="evening"), "hora"] = df.hora.astype("int")+12
    df.loc[(df.part_day=="afternoon")&(df.hora!="12"), "hora"] = df.hora.astype("int")+12
    df.loc[(df.part_day=="midnight"), "hora"] = df.hora.astype("int")+12
    df.loc[((df.part_day=="night")&(df.hora.astype("int")>=7)), "hora"] = df.hora.astype("int")+12

df=df.drop(['time','part_day','log_count'], axis = 1)


In [5]:
# Escribir la tabla en un csv
df.to_csv('output.csv', index = False, sep = "|")


In [6]:
#Proposed KPIs
# Most repeated words 
# Messages by hour
# Messages by date
# Day with more messages
# Total number of messages
# number of pictures
# One Image
# Number of meesages by sender
# Filter by month, sender and hora


In [7]:
#QA tests
df["base"].str.split("-", expand=True)[0].str.split().str[-1].unique() # Para la parte del día
df["base"].str.split("-", expand=True)[1].str.split(":", expand=True)[0].unique() # Para remitente


array([' Yo', ' Mi Amor'], dtype=object)